In [1]:
from shapely.geometry import Point, mapping, LineString
import geopandas as gpd
import pandas as pd
import folium
from folium import PolyLine
from folium.plugins import MarkerCluster, Fullscreen
from folium.features import CustomIcon
from branca.element import MacroElement
from jinja2 import Template
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# read your individual csv here...find yours idk where that thang be
apparc_counties = pd.read_csv("../data/Appalachian-Counties-Served-by-ARC_2021.csv")
fema_areas = pd.read_csv("../data/FemaWebDeclarationAreas.csv")
fema_disasters = pd.read_csv("../data/FemaWebDisasterDeclarations.csv")
activeMining91 = gpd.read_file("../data/MTM_Shapefile/Shapefile/4326/1991_activeMining_4326.shp").to_crs(epsg=4326)

In [3]:
fema_areas['COUNTY'] = fema_areas['COUNTY'].str[:-9]

In [4]:
cleaned_county = pd.merge(apparc_counties, fema_areas, on='COUNTY', how='inner')

In [5]:
df = pd.merge(fema_disasters, cleaned_county, on='disasterNumber', how='inner')

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
cleaned_county

,FIPS,STATE,COUNTY,id,disasterNumber,programTypeCode,programTypeDescription,stateCode,stateName,placeCode,designatedDate,entryDate,updateDate,closeoutDate,hash,lastRefresh
0,1007,Alabama,Bibb,084d5898-3410-4705-a5df-00bba2d5ad78,1971,CC,CC - Crisis Counseling,AL,Alabama,99007,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2024-05-02T00:00:00.000Z,03ca639b0515dddbb6bb6047ef8bc8acb7a73083,2024-05-02T14:41:44.963Z
1,1007,Alabama,Bibb,71a0618b-2b8c-4063-b20b-a3be20c7fd30,1971,DFA,DFA - Direct Federal Assistance,AL,Alabama,99007,2011-04-28T00:00:00.000Z,2011-04-29T00:00:00.000Z,2011-04-29T00:00:00.000Z,2024-05-02T00:00:00.000Z,8277df68d4e51413b18a9eead5208c9e71a642e9,2024-05-02T14:41:44.963Z
2,1007,Alabama,Bibb,6e418000-5743-4f18-bb23-f7e939fc4253,1971,DUA,DUA - Disaster Unemployment Assist.,AL,Alabama,99007,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2024-05-02T00:00:00.000Z,43316491dd15afab060556cdcf95203f89f2f97a,2024-05-02T14:41:44.963Z
3,1007,Alabama,Bibb,64ace98e-7254-44f3-9d99-b65c0c5306ee,1971,HA,HA - Housing Assistance,AL,Alabama,99007,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2011-05-01T00:00:00.000Z,2024-05-02T00:00:00.000Z,b838210ddf343effd3f971ef0b6dfe1db192eff8,2024-05-02T14:41:44.963Z
4,1007,Alabama,Bibb,7b8bc1f4-126e-41c2-be8d-82bc74c81767,1971,HM,HM - Hazard Mitigation,AL,Alabama,99007,2011-04-28T00:00:00.000Z,2011-04-29T00:00:00.000Z,2011-04-29T00:00:00.000Z,2024-05-02T00:00:00.000Z,f03c7eff2506d220d758cb62c977bf5428f5b651,2024-05-02T14:41:44.963Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372546,54109,West Virginia,Wyoming,220b6a2a-a266-43e5-9ec5-b53c0419ca62,1115,PA-C,PA-C - Roads & Bridges,WV,West Virginia,99109,1996-05-23T00:00:00.000Z,1996-05-29T00:00:00.000Z,1996-05-29T00:00:00.000Z,2006-03-08T00:00:00.000Z,23319b9dca013d4e909af41dcda2ae922b87d518,2023-09-28T23:41:27.459Z
372547,54109,West Virginia,Wyoming,3958bbd2-51b1-4f28-94d0-0bf436f2643b,1115,PA-D,PA-D - Water Control Facilities,WV,West Virginia,99109,1996-05-23T00:00:00.000Z,1996-05-29T00:00:00.000Z,1996-05-29T00:00:00.000Z,2006-03-08T00:00:00.000Z,50bf2a5e61a019b4cfc41769b165722bcfc0bae8,2023-09-28T23:41:27.459Z
372548,54109,West Virginia,Wyoming,d5ceb88d-d37e-495b-bc16-d7e1fc1002ca,1115,PA-E,PA-E - Public Buildings,WV,West Virginia,99109,1996-05-23T00:00:00.000Z,1996-05-29T00:00:00.000Z,1996-05-29T00:00:00.000Z,2006-03-08T00:00:00.000Z,54e8b814f3e3cd9909f8862dcc9857312d81b909,2023-09-28T23:41:27.459Z
372549,54109,West Virginia,Wyoming,0adcbc13-ea4a-4757-8827-cda83e7a0d48,1115,PA-F,PA-F - Public Utilities,WV,West Virginia,99109,1996-05-23T00:00:00.000Z,1996-05-29T00:00:00.000Z,1996-05-29T00:00:00.000Z,2006-03-08T00:00:00.000Z,1556e7f8a540e05f367cdf59c5160e561cb790a0,2023-09-28T23:41:27.459Z


In [8]:
df = df[df['COUNTY'].notna() & (df['COUNTY'] != '')]

In [9]:
flood_df = df[df['incidentType'] == 'Flood']
flood_df

,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,stateCode_x,stateName_x,incidentType,entryDate_x,updateDate_x,closeoutDate_x,region,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,designatedIncidentTypes,declarationRequestDate,disasterPageUrl,shapefileUrl,kmzfileUrl,geoJsonUrl,id_x,hash_x,lastRefresh_x,FIPS,STATE,COUNTY,id_y,programTypeCode,programTypeDescription,stateCode_y,stateName_y,placeCode,designatedDate,entryDate_y,updateDate_y,closeoutDate_y,hash_y,lastRefresh_y
5161,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,Alabama,Calhoun,c979e885-e39b-4a08-b65f-d116f3259dce,CC,CC - Crisis Counseling,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,3b29f545845900bec7f76bff3394e3ba5ef83358,2024-05-22T21:02:10.095Z
5162,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,Alabama,Calhoun,ea1e5d71-403e-4287-9ed2-8544270a956d,DUA,DUA - Disaster Unemployment Assist.,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,9ed5cdf91b46a4fbf8f98ec038b68659d825bf96,2024-05-22T21:02:10.095Z
5163,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,Alabama,Calhoun,26f2fcff-9e2f-4295-ba04-244d4e64ef11,HA,HA - Housing Assistance,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,dfc0185822c8aca2091764961844c180af666e58,2024-05-22T21:02:10.095Z
5164,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,Alabama,Calhoun,076dfc33-2eb9-4953-afcb-39887fb6a23a,HM,HM - Hazard Mitigation,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,493812839ad67dd719f104b250512f5260c74e4d,2024-05-22T21:02:10.095Z
5165,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_478

In [10]:
flood_pct_ttl = (len(flood_df))/(len(df))*100
flood_pct_ttl

23.324054961602574

In [11]:
apparc_counties['COUNTY'] = apparc_counties['COUNTY'].str.strip().str.upper()
apparc_counties['STATE'] = apparc_counties['STATE'].str.strip().str.upper()

flood_df['COUNTY'] = flood_df['COUNTY'].str.strip().str.upper()
flood_df['STATE'] = flood_df['STATE'].str.strip().str.upper()

target_pairs = set(zip(apparc_counties['STATE'], apparc_counties['COUNTY']))
filtered_df = flood_df[flood_df.apply(lambda row: (row['STATE'], row['COUNTY']) in target_pairs, axis=1)]

C:\Users\jcbja\AppData\Local\Temp\ipykernel_44448\2052150048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flood_df['COUNTY'] = flood_df['COUNTY'].str.strip().str.upper()
C:\Users\jcbja\AppData\Local\Temp\ipykernel_44448\2052150048.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flood_df['STATE'] = flood_df['STATE'].str.strip().str.upper()


In [24]:
county_state_list = [
    ('Bibb', 'AL'),
    ('Blount', 'AL'),
    ('Calhoun', 'AL'),
    ('Chambers', 'AL'),
    ('Cherokee', 'AL'),
    ('Chilton', 'AL'),
    ('Clay', 'AL'),
    ('Cleburne', 'AL'),
    ('Colbert', 'AL'),
    ('Coosa', 'AL'),
    ('Cullman', 'AL'),
    ('De Kalb', 'AL'),
    ('Elmore', 'AL'),
    ('Etowah', 'AL'),
    ('Fayette', 'AL'),
    ('Franklin', 'AL'),
    ('Hale', 'AL'),
    ('Jackson', 'AL'),
    ('Jefferson', 'AL'),
    ('Lamar', 'AL'),
    ('Lauderdale', 'AL'),
    ('Lawrence', 'AL'),
    ('Limestone', 'AL'),
    ('Macon', 'AL'),
    ('Madison', 'AL'),
    ('Marion', 'AL'),
    ('Marshall', 'AL'),
    ('Morgan', 'AL'),
    ('Pickens', 'AL'),
    ('Randolph', 'AL'),
    ('St. Clair', 'AL'),
    ('Shelby', 'AL'),
    ('Talladega', 'AL'),
    ('Tallapoosa', 'AL'),
    ('Tuscaloosa', 'AL'),
    ('Walker', 'AL'),
    ('Winston', 'AL'),
    ('Banks', 'GA'),
    ('Barrow', 'GA'),
    ('Bartow', 'GA'),
    ('Carroll', 'GA'),
    ('Catoosa', 'GA'),
    ('Chattooga', 'GA'),
    ('Cherokee', 'GA'),
    ('Dade', 'GA'),
    ('Dawson', 'GA'),
    ('Douglas', 'GA'),
    ('Elbert', 'GA'),
    ('Fannin', 'GA'),
    ('Floyd', 'GA'),
    ('Forsyth', 'GA'),
    ('Franklin', 'GA'),
    ('Gilmer', 'GA'),
    ('Gordon', 'GA'),
    ('Gwinnett', 'GA'),
    ('Habersham', 'GA'),
    ('Hall', 'GA'),
    ('Haralson', 'GA'),
    ('Hart', 'GA'),
    ('Heard', 'GA'),
    ('Jackson', 'GA'),
    ('Lumpkin', 'GA'),
    ('Madison', 'GA'),
    ('Murray', 'GA'),
    ('Paulding', 'GA'),
    ('Pickens', 'GA'),
    ('Polk', 'GA'),
    ('Rabun', 'GA'),
    ('Stephens', 'GA'),
    ('Towns', 'GA'),
    ('Union', 'GA'),
    ('Walker', 'GA'),
    ('White', 'GA'),
    ('Whitfield', 'GA'),
    ('Adair', 'KY'),
    ('Bath', 'KY'),
    ('Bell', 'KY'),
    ('Boyd', 'KY'),
    ('Breathitt', 'KY'),
    ('Carter', 'KY'),
    ('Casey', 'KY'),
    ('Clark', 'KY'),
    ('Clay', 'KY'),
    ('Clinton', 'KY'),
    ('Cumberland', 'KY'),
    ('Edmonson', 'KY'),
    ('Elliott', 'KY'),
    ('Estill', 'KY'),
    ('Fleming', 'KY'),
    ('Floyd', 'KY'),
    ('Garrard', 'KY'),
    ('Green', 'KY'),
    ('Greenup', 'KY'),
    ('Harlan', 'KY'),
    ('Hart', 'KY'),
    ('Jackson', 'KY'),
    ('Johnson', 'KY'),
    ('Knott', 'KY'),
    ('Knox', 'KY'),
    ('Laurel', 'KY'),
    ('Lawrence', 'KY'),
    ('Lee', 'KY'),
    ('Leslie', 'KY'),
    ('Letcher', 'KY'),
    ('Lewis', 'KY'),
    ('Lincoln', 'KY'),
    ('McCreary', 'KY'),
    ('Madison', 'KY'),
    ('Magoffin', 'KY'),
    ('Martin', 'KY'),
    ('Menifee', 'KY'),
    ('Metcalfe', 'KY'),
    ('Monroe', 'KY'),
    ('Montgomery', 'KY'),
    ('Morgan', 'KY'),
    ('Nicholas', 'KY'),
    ('Owsley', 'KY'),
    ('Perry', 'KY'),
    ('Pike', 'KY'),
    ('Powell', 'KY'),
    ('Pulaski', 'KY'),
    ('Robertson', 'KY'),
    ('Rockcastle', 'KY'),
    ('Rowan', 'KY'),
    ('Russell', 'KY'),
    ('Wayne', 'KY'),
    ('Whitley', 'KY'),
    ('Wolfe', 'KY'),
    ('Allegany', 'MD'),
    ('Garrett', 'MD'),
    ('Washington', 'MD'),
    ('Alcorn', 'MS'),
    ('Benton', 'MS'),
    ('Calhoun', 'MS'),
    ('Chickasaw', 'MS'),
    ('Choctaw', 'MS'),
    ('Clay', 'MS'),
    ('Itawamba', 'MS'),
    ('Kemper', 'MS'),
    ('Lee', 'MS'),
    ('Lowndes', 'MS'),
    ('Marshall', 'MS'),
    ('Monroe', 'MS'),
    ('Montgomery', 'MS'),
    ('Noxubee', 'MS'),
    ('Oktibbeha', 'MS'),
    ('Panola', 'MS'),
    ('Pontotoc', 'MS'),
    ('Prentiss', 'MS'),
    ('Tippah', 'MS'),
    ('Tishomingo', 'MS'),
    ('Union', 'MS'),
    ('Webster', 'MS'),
    ('Winston', 'MS'),
    ('Yalobusha', 'MS'),
    ('Allegany', 'NY'),
    ('Broome', 'NY'),
    ('Cattaraugus', 'NY'),
    ('Chautauqua', 'NY'),
    ('Chemung', 'NY'),
    ('Chenango', 'NY'),
    ('Cortland', 'NY'),
    ('Delaware', 'NY'),
    ('Otsego', 'NY'),
    ('Schoharie', 'NY'),
    ('Schuyler', 'NY'),
    ('Steuben', 'NY'),
    ('Tioga', 'NY'),
    ('Tompkins', 'NY'),
    ('Alexander', 'NC'),
    ('Alleghany', 'NC'),
    ('Ashe', 'NC'),
    ('Avery', 'NC'),
    ('Buncombe', 'NC'),
    ('Burke', 'NC'),
    ('Caldwell', 'NC'),
    ('Catawba', 'NC'),
    ('Cherokee', 'NC'),
    ('Clay', 'NC'),
    ('Cleveland', 'NC'),
    ('Davie', 'NC'),
    ('Forsyth', 'NC'),
    ('Graham', 'NC'),
    ('Haywood', 'NC'),
    ('Henderson', 'NC'),
    ('Jackson', 'NC'),
    ('McDowell', 'NC'),
    ('Macon', 'NC'),
    ('Madison', 'NC'),
    ('Mitchell', 'NC'),
    ('Polk', 'NC'),
    ('Rutherford', 'NC'),
    ('Stokes', 'NC'),
    ('Surry', 'NC'),
    ('Swain', 'NC'),
    ('Transylvania', 'NC'),
    ('Watauga', 'NC'),
    ('Wilkes', 'NC'),
    ('Yadkin', 'NC'),
    ('Yancey', 'NC'),
    ('Adams', 'OH'),
    ('Ashtabula', 'OH'),
    ('Athens', 'OH'),
    ('Belmont', 'OH'),
    ('Brown', 'OH'),
    ('Carroll', 'OH'),
    ('Clermont', 'OH'),
    ('Columbiana', 'OH'),
    ('Coshocton', 'OH'),
    ('Gallia', 'OH'),
    ('Guernsey', 'OH'),
    ('Harrison', 'OH'),
    ('Highland', 'OH'),
    ('Hocking', 'OH'),
    ('Holmes', 'OH'),
    ('Jackson', 'OH'),
    ('Jefferson', 'OH'),
    ('Lawrence', 'OH'),
    ('Mahoning', 'OH'),
    ('Meigs', 'OH'),
    ('Monroe', 'OH'),
    ('Morgan', 'OH'),
    ('Muskingum', 'OH'),
    ('Noble', 'OH'),
    ('Perry', 'OH'),
    ('Pike', 'OH'),
    ('Ross', 'OH'),
    ('Scioto', 'OH'),
    ('Trumbull', 'OH'),
    ('Tuscarawas', 'OH'),
    ('Vinton', 'OH'),
    ('Washington', 'OH'),
    ('Allegheny', 'PA'),
    ('Armstrong', 'PA'),
    ('Beaver', 'PA'),
    ('Bedford', 'PA'),
    ('Blair', 'PA'),
    ('Bradford', 'PA'),
    ('Butler', 'PA'),
    ('Cambria', 'PA'),
    ('Cameron', 'PA'),
    ('Carbon', 'PA'),
    ('Centre', 'PA'),
    ('Clarion', 'PA'),
    ('Clearfield', 'PA'),
    ('Clinton', 'PA'),
    ('Columbia', 'PA'),
    ('Crawford', 'PA'),
    ('Elk', 'PA'),
    ('Erie', 'PA'),
    ('Fayette', 'PA'),
    ('Forest', 'PA'),
    ('Fulton', 'PA'),
    ('Greene', 'PA'),
    ('Huntingdon', 'PA'),
    ('Indiana', 'PA'),
    ('Jefferson', 'PA'),
    ('Juniata', 'PA'),
    ('Lackawanna', 'PA'),
    ('Lawrence', 'PA'),
    ('Luzerne', 'PA'),
    ('Lycoming', 'PA'),
    ('McKean', 'PA'),
    ('Mercer', 'PA'),
    ('Mifflin', 'PA'),
    ('Monroe', 'PA'),
    ('Montour', 'PA'),
    ('Northumberland', 'PA'),
    ('Perry', 'PA'),
    ('Pike', 'PA'),
    ('Potter', 'PA'),
    ('Schuylkill', 'PA'),
    ('Snyder', 'PA'),
    ('Somerset', 'PA'),
    ('Sullivan', 'PA'),
    ('Susquehanna', 'PA'),
    ('Tioga', 'PA'),
    ('Union', 'PA'),
    ('Venango', 'PA'),
    ('Warren', 'PA'),
    ('Washington', 'PA'),
    ('Wayne', 'PA'),
    ('Westmoreland', 'PA'),
    ('Wyoming', 'PA'),
    ('Anderson', 'SC'),
    ('Cherokee', 'SC'),
    ('Greenville', 'SC'),
    ('Oconee', 'SC'),
    ('Pickens', 'SC'),
    ('Spartanburg', 'SC'),
    ('Union', 'SC'),
    ('Anderson', 'TN'),
    ('Bledsoe', 'TN'),
    ('Blount', 'TN'),
    ('Bradley', 'TN'),
    ('Campbell', 'TN'),
    ('Cannon', 'TN'),
    ('Carter', 'TN'),
    ('Claiborne', 'TN'),
    ('Clay', 'TN'),
    ('Cocke', 'TN'),
    ('Coffee', 'TN'),
    ('Cumberland', 'TN'),
    ('De Kalb', 'TN'),
    ('Fentress', 'TN'),
    ('Franklin', 'TN'),
    ('Grainger', 'TN'),
    ('Greene', 'TN'),
    ('Grundy', 'TN'),
    ('Hamblen', 'TN'),
    ('Hamilton', 'TN'),
    ('Hancock', 'TN'),
    ('Hawkins', 'TN'),
    ('Jackson', 'TN'),
    ('Jefferson', 'TN'),
    ('Johnson', 'TN'),
    ('Knox', 'TN'),
    ('Lawrence', 'TN'),
    ('Lewis', 'TN'),
    ('Loudon', 'TN'),
    ('McMinn', 'TN'),
    ('Macon', 'TN'),
    ('Marion', 'TN'),
    ('Meigs', 'TN'),
    ('Monroe', 'TN'),
    ('Morgan', 'TN'),
    ('Overton', 'TN'),
    ('Pickett', 'TN'),
    ('Polk', 'TN'),
    ('Putnam', 'TN'),
    ('Rhea', 'TN'),
    ('Roane', 'TN'),
    ('Scott', 'TN'),
    ('Sequatchie', 'TN'),
    ('Sevier', 'TN'),
    ('Smith', 'TN'),
    ('Sullivan', 'TN'),
    ('Unicoi', 'TN'),
    ('Union', 'TN'),
    ('Van Buren', 'TN'),
    ('Warren', 'TN'),
    ('Washington', 'TN'),
    ('White', 'TN'),
    ('Alleghany + Covington city', 'VA'),
    ('Bath', 'VA'),
    ('Bland', 'VA'),
    ('Botetourt', 'VA'),
    ('Buchanan', 'VA'),
    ('Carroll + Galax city', 'VA'),
    ('Craig', 'VA'),
    ('Dickenson', 'VA'),
    ('Floyd', 'VA'),
    ('Giles', 'VA'),
    ('Grayson', 'VA'),
    ('Henry + Martinsville city', 'VA'),
    ('Highland', 'VA'),
    ('Lee', 'VA'),
    ('Montgomery + Radford city', 'VA'),
    ('Patrick', 'VA'),
    ('Pulaski', 'VA'),
    ('Rockbridge + Buena Vista city + Lexington city', 'VA'),
    ('Russell', 'VA'),
    ('Scott', 'VA'),
    ('Smyth', 'VA'),
    ('Tazewell', 'VA'),
    ('Washington + Bristol city', 'VA'),
    ('Wise + Norton city', 'VA'),
    ('Wythe', 'VA'),
    ('Barbour', 'WV'),
    ('Berkeley', 'WV'),
    ('Boone', 'WV'),
    ('Braxton', 'WV'),
    ('Brooke', 'WV'),
    ('Cabell', 'WV'),
    ('Calhoun', 'WV'),
    ('Clay', 'WV'),
    ('Doddridge', 'WV'),
    ('Fayette', 'WV'),
    ('Gilmer', 'WV'),
    ('Grant', 'WV'),
    ('Greenbrier', 'WV'),
    ('Hampshire', 'WV'),
    ('Hancock', 'WV'),
    ('Hardy', 'WV'),
    ('Harrison', 'WV'),
    ('Jackson', 'WV'),
    ('Jefferson', 'WV'),
    ('Kanawha', 'WV'),
    ('Lewis', 'WV'),
    ('Lincoln', 'WV'),
    ('Logan', 'WV'),
    ('McDowell', 'WV'),
    ('Marion', 'WV'),
    ('Marshall', 'WV'),
    ('Mason', 'WV'),
    ('Mercer', 'WV'),
    ('Mineral', 'WV'),
    ('Mingo', 'WV'),
    ('Monongalia', 'WV'),
    ('Monroe', 'WV'),
    ('Morgan', 'WV'),
    ('Nicholas', 'WV'),
    ('Ohio', 'WV'),
    ('Pendleton', 'WV'),
    ('Pleasants', 'WV'),
    ('Pocahontas', 'WV'),
    ('Preston', 'WV'),
    ('Putnam', 'WV'),
    ('Raleigh', 'WV'),
    ('Randolph', 'WV'),
    ('Ritchie', 'WV'),
    ('Roane', 'WV'),
    ('Summers', 'WV'),
    ('Taylor', 'WV'),
    ('Tucker', 'WV'),
    ('Tyler', 'WV'),
    ('Upshur', 'WV'),
    ('Wayne', 'WV'),
    ('Webster', 'WV'),
    ('Wetzel', 'WV'),
    ('Wirt', 'WV'),
    ('Wood', 'WV'),
    ('Wyoming', 'WV'),
]


# Make a new column combining 'county' and 'state' columns
df['county_state_list'] = df['COUNTY'].str.strip() + ', ' + df['STATE'].str.strip()

# Build the set of allowed "county, state" strings
allowed_county_state = {f"{county.strip()}, {state.strip()}" for county, state in county_state_list}

# Keep only rows where 'county_state' is in the allowed set
filtered_df = df[df['county_state_list'].isin(allowed_county_state)].copy()

In [48]:
filtered_df

,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,stateCode_x,stateName_x,incidentType,entryDate_x,updateDate_x,closeoutDate_x,region,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,designatedIncidentTypes,declarationRequestDate,disasterPageUrl,shapefileUrl,kmzfileUrl,geoJsonUrl,id_x,hash_x,lastRefresh_x,FIPS,STATE,COUNTY,id_y,programTypeCode,programTypeDescription,stateCode_y,stateName_y,placeCode,designatedDate,entryDate_y,updateDate_y,closeoutDate_y,hash_y,lastRefresh_y
5161,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,c979e885-e39b-4a08-b65f-d116f3259dce,CC,CC - Crisis Counseling,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,3b29f545845900bec7f76bff3394e3ba5ef83358,2024-05-22T21:02:10.095Z
5162,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,ea1e5d71-403e-4287-9ed2-8544270a956d,DUA,DUA - Disaster Unemployment Assist.,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,9ed5cdf91b46a4fbf8f98ec038b68659d825bf96,2024-05-22T21:02:10.095Z
5163,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,26f2fcff-9e2f-4295-ba04-244d4e64ef11,HA,HA - Housing Assistance,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,dfc0185822c8aca2091764961844c180af666e58,2024-05-22T21:02:10.095Z
5164,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,076dfc33-2eb9-4953-afcb-39887fb6a23a,HM,HM - Hazard Mitigation,TX,Texas,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,493812839ad67dd719f104b250512f5260c74e4d,2024-05-22T21:02:10.095Z
5165,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,TX,Texas,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_478

In [49]:
filtered_df1 = filtered_df1.drop(['stateCode_x', 'stateName_x', 'stateCode_y', 'stateName_y'], axis=1)

In [50]:
filtered_df1

,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,incidentType,entryDate_x,updateDate_x,closeoutDate_x,region,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,designatedIncidentTypes,declarationRequestDate,disasterPageUrl,shapefileUrl,kmzfileUrl,geoJsonUrl,id_x,hash_x,lastRefresh_x,FIPS,STATE,COUNTY,id_y,programTypeCode,programTypeDescription,placeCode,designatedDate,entryDate_y,updateDate_y,closeoutDate_y,hash_y,lastRefresh_y
5161,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,c979e885-e39b-4a08-b65f-d116f3259dce,CC,CC - Crisis Counseling,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,3b29f545845900bec7f76bff3394e3ba5ef83358,2024-05-22T21:02:10.095Z
5162,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,ea1e5d71-403e-4287-9ed2-8544270a956d,DUA,DUA - Disaster Unemployment Assist.,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,9ed5cdf91b46a4fbf8f98ec038b68659d825bf96,2024-05-22T21:02:10.095Z
5163,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,26f2fcff-9e2f-4295-ba04-244d4e64ef11,HA,HA - Housing Assistance,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,dfc0185822c8aca2091764961844c180af666e58,2024-05-22T21:02:10.095Z
5164,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc27,340e565d879241a94d4475d18818ddff8e483314,2025-03-11T19:21:45.366Z,1015,ALABAMA,CALHOUN,076dfc33-2eb9-4953-afcb-39887fb6a23a,HM,HM - Hazard Mitigation,99057,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,2024-05-22T00:00:00.000Z,NaN,493812839ad67dd719f104b250512f5260c74e4d,2024-05-22T21:02:10.095Z
5165,4781,2024-05-17T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",2024-04-26T00:00:00.000Z,2024-06-05T00:00:00.000Z,Major Disaster,Flood,2024-05-17T00:00:00.000Z,2024-09-23T00:00:00.000Z,NaN,6,1.0,0.0,1.0,1.0,W,2024-05-15T00:00:00.000Z,https://www.fema.gov/disaster/4781,https://gis.fema.gov/maps/dec_4781.zip,https://gis.fema.gov/maps/dec_4781.kmz,https://gis.fema.gov/geoserver/FEMA/ows?servic...,fcf9b350-3993-49d9-ab26-13e79988cc